# **☘️ Wadhwani AIBollworm CLASSIFICATION USING RESNET-9 ☘️**

## **Description of the dataset 📝**

Data has been collected from farmers since 2018. Some of that data was collected using a dedicated data collection app. Most of it, however, was collected from the app itself; making the dataset unique among other farm pest datasets. There are approximately 13 000 images.

Positive images are indicated by a 1 in the training set. These are images that contain at least one bollworm on an insect card. Negative images are images that do not have a bounding box and cannot be used to count the number of bollworms per card.

## **Our goal 🎯**

The objective of this competition is to create a machine learning model that classifies images as positive (contains bollworm moths) or negative (no bollworm moths). This model will form one part of the greater solution to control bollworm pests in India delivered by the Wadhwani AI app.


# **Setup**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade -q fastai
!pip install timm -q
!pip install albumentations==0.4.6 -q
!pip install transformers -q

     |████████████████████████████████| 548 kB 38.3 MB/s 
     |████████████████████████████████| 163 kB 73.5 MB/s 
     |████████████████████████████████| 117 kB 32.9 MB/s 
     |████████████████████████████████| 4.9 MB 28.6 MB/s 
     |████████████████████████████████| 6.6 MB 62.5 MB/s 


# **Libraries**

In [ ]:
from fastai.vision.all import *
import timm
import warnings 
import albumentations as A
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold

import gc
import numpy as np
import inspect
from fastai.metrics import accuracy
from fastai.optimizer import OptimWrapper
from torch import optim
from functools import partial
from transformers import AdamW
from google.colab import files

warnings.filterwarnings('ignore')
set_seed(21, reproducible=True)

# **Environment configuration**

In [ ]:
PATHS = {
    'arch1': '/content/drive/MyDrive/WadhwaniAIBollwormClassification/convnext_tiny/',
    'raw': '/content/drive/MyDrive/WadhwaniAIBollwormClassification/',
    'images': '/content/drive/MyDrive/WadhwaniAIBollwormClassification/Images/'}

os.makedirs(PATHS['arch1'], exist_ok=True)

In [ ]:
seed = 2022 # for reproductibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available(): 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# **🧭Dataset🧭**

In [ ]:
train = pd.read_csv(PATHS["raw"] + 'Train.csv')
test = pd.read_csv(PATHS["raw"]  + 'Test.csv')


columns = ["image_id", "Label"]
train.columns = columns

train.head()

,image_id,Label
0,id_006296f82479c03459102436.jpg,0
1,id_006e9f771965fd78278baa19.jpg,1
2,id_00750938fb12fad4fd8fad24.jpg,1
3,id_00773a42b1415d887107663a.jpg,1
4,id_00c6d6db61a9d7393ad39186.jpg,1


In [ ]:
Train = train.copy()
Test = test.copy()

# add image paths
Train['image_path'] = PATHS["images"] + Train.image_id
Test['image_path'] = PATHS["images"] + Test.image_id

Train = Train[['image_path', 'Label']]
Test = Test[['image_path']]


skf = StratifiedKFold(10, shuffle=True, random_state=21)
X = Train.drop(columns='Label')
y = Train.Label

for fold, (_, valid_index) in enumerate(skf.split(X, y)):
  Train.loc[valid_index, "fold"] = fold

Train.fold = Train.fold.astype(int)

Train.head(2)

,image_path,Label,fold
0,/content/drive/MyDrive/WadhwaniAIBollwormClassification/Images/id_006296f82479c03459102436.jpg,0,4
1,/content/drive/MyDrive/WadhwaniAIBollwormClassification/Images/id_006e9f771965fd78278baa19.jpg,1,2


# **👷 Utility Functions and Model Architechture 👷**

In [ ]:
IMG_HEIGHT = IMG_WIDTH = 224
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
INCEPTION_MEAN = INCEPTION_STD = (0.5, 0.5, 0.5)


class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        
        return aug_img


def get_train_transforms(mean_std):
    augmentations = [
        A.HorizontalFlip(p=.5),
        A.VerticalFlip(p=.5),
        A.RandomRotate90(p=.5),
        A.ImageCompression(quality_lower=99, quality_upper=100),
        A.ShiftScaleRotate(
            shift_limit=0.2, scale_limit=0.2, 
            rotate_limit=45, border_mode=0, p=.5
        ),
        A.Resize(IMG_HEIGHT, IMG_WIDTH),
        A.Cutout(
            max_h_size=int(IMG_HEIGHT*0.4),
            max_w_size=int(IMG_WIDTH*0.4),
            num_holes=1,
            p=.75,
        ),
    ]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)


def get_valid_transforms(mean_std):
    augmentations = [A.Resize(IMG_HEIGHT, IMG_WIDTH)]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)

def get_item_tfms(mean_std='imagenet'):
    return [AlbumentationsTransform(get_train_transforms(mean_std), get_valid_transforms(mean_std))]

In [ ]:
class Head(nn.Module):
    def __init__(self, in_features, out_features):
        super(Head, self).__init__()
        self.head = nn.Linear(in_features=in_features, out_features=out_features)

    def forward(self, x):
        return self.head(x)


class Net(nn.Module):
    def __init__(
        self,
        base_model="resnet50",
        pretrained=True,
        checkpoint_path=None,
        num_classes=2,
    ):
        super(Net, self).__init__()

        self.backbone = timm.create_model(
            base_model, pretrained=pretrained, checkpoint_path=checkpoint_path
        )
        in_features = self.backbone.get_classifier().in_features
        self.backbone.reset_classifier(num_classes=0, global_pool="avg")
        self.neck = Head(in_features=in_features, out_features=in_features)
        self.head = Head(in_features=in_features, out_features=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.neck(x)
        x = self.head(x)
        x = F.log_softmax(x, dim=1)
        return x


In [ ]:
roc = RocAucBinary()

In [ ]:
def free_memory(to_delete: list):
    calling_namespace = inspect.currentframe().f_back

    for _var in to_delete:
        calling_namespace.f_locals.pop(_var, None)
        gc.collect()
        torch.cuda.empty_cache()

def get_dls(fold=0, mean_std='imagenet'):
    clas_block = DataBlock(blocks=(ImageBlock, CategoryBlock),
                        splitter=MaskSplitter(Train.fold == fold),
                        get_x=ColReader("image_path"),
                        get_y=ColReader("Label"),
                        item_tfms=get_item_tfms(mean_std),
                        )
    dls = clas_block.dataloaders(Train, bs=32)
    dls.rng.seed(21)

    return dls

# **🏋️ Training the model 🏋️**


In [ ]:
predictions = []
for fold in [1]:
    print('*'*25+f"Fold {fold}"+'*'*25)
    m_name = f'{PATHS["arch1"]}fold-{fold}'
    dls = get_dls(fold, 'imagenet')
    learn = Learner(dls, Net(base_model='convnext_tiny'), 
                    loss_func=CrossEntropyLossFlat(), 
                    opt_func=partial(OptimWrapper, opt=optim.Adam), 
                    cbs=[SaveModelCallback(reset_on_fit=False, fname=m_name)],
                    metrics=[accuracy, error_rate, roc]
            )
    
    learn.fit_one_cycle(15, 2e-4)
    learn.save(m_name+'_stage-1')
    learn = learn.load(m_name)
    learn.freeze_to(-5)
    learn.fit_one_cycle(4, 2e-4)
    learn.save(m_name+'_stage-2');


    model = learn.load(m_name)
    preds, _ = model.tta(dl=dls.test_dl(Test), n=4)
    predictions.append(preds)

    free_memory([model, learn, preds, dls])

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth


epoch,train_loss,valid_loss,accuracy,error_rate,roc_auc_score,time
0,0.549959,0.448349,0.830769,0.169231,0.856095,19:44
1,0.361194,0.208906,0.928205,0.071795,0.968521,12:06
2,0.271947,0.204777,0.933333,0.066667,0.962722,11:56
3,0.231595,0.213494,0.933333,0.066667,0.975740,11:48
4,0.194261,0.209958,0.928205,0.071795,0.968047,12:08
5,0.163223,0.180135,0.943590,0.056410,0.974911,11:50
6,0.126704,0.224010,0.938462,0.061538,0.971716,12:18
7,0.115792,0.173847,0.948718,0.051282,0.978817,12:10
8,0.089565,0.209649,0.943590,0.056410,0.974793,12:23
9,0.094046,0.218597,0.938462,0.061538,0.975030,12:02


Better model found at epoch 0 with valid_loss value: 0.4483489990234375.
Better model found at epoch 1 with valid_loss value: 0.20890553295612335.
Better model found at epoch 2 with valid_loss value: 0.20477737486362457.
Better model found at epoch 5 with valid_loss value: 0.18013514578342438.
Better model found at epoch 7 with valid_loss value: 0.17384693026542664.


epoch,train_loss,valid_loss,accuracy,error_rate,roc_auc_score,time
0,0.063855,0.207429,0.948718,0.051282,0.977929,11:59
1,0.074338,0.213276,0.948718,0.051282,0.975858,11:47


epoch,train_loss,valid_loss,accuracy,error_rate,roc_auc_score,time
0,0.063855,0.207429,0.948718,0.051282,0.977929,11:59
1,0.074338,0.213276,0.948718,0.051282,0.975858,11:47
2,0.066059,0.222147,0.948718,0.051282,0.975148,11:56


In [ ]:
test = pd.read_csv(PATHS["raw"] + 'Test.csv')
pred = np.mean([i.numpy() for j, i in enumerate(predictions[-1:])], axis=0)

submission = pd.DataFrame({'Image_ID': test['image_id']})
submission["Label"] = [x[1] for x in pred]

submission.to_csv("FinalSubmission.csv", index=False)
files.download(f"FinalSubmission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
submission

,Image_ID,Label
0,id_005102f664b820f778291dee.jpg,0.999953
1,id_0066456f5fb2cd858c69ab39.jpg,0.996827
2,id_007159c1fa015ba6f394deeb.jpg,0.004922
3,id_00ba116c0f45a71a7e0e652c.jpg,0.017287
4,id_00c11f7689e2351305cb12e3.jpg,0.850259
...,...,...
835,id_fec06cb1f998a96524bdcbb3.jpg,0.999622
836,id_ffad8f3773a4222f8fe5ba1a.jpg,0.956714
837,id_ffb65e6de900c49d8f2ef95a.jpg,0.999879
838,id_ffc0e41e10b0c964d4a02811.jpg,0.014591
